<a href="https://colab.research.google.com/github/AareanaReza/CS598-DLH-Final-Project/blob/main/CS598ProjectCodeCommands.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#CS 598 DLH Project: Diagnoses Prediction

###Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

###Paper and Github Citations

Zaghir J, Rodrigues-Jr JF, Goeuriot L, Amer-Yahia S. Real-world Patient Trajectory Prediction from Clinical Notes Using Artificial Neural Networks and UMLS-Based Extraction of Concepts. J Healthc Inform Res. 2021 Jun 5;5(4):474-496. doi: 10.1007/s41666-021-00100-z. PMID: 35419508; PMCID: PMC8982755.
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8982755/#CR22
GitHub:https://github.com/JamilProg/patient_trajectory_prediction




###Dependencies

In [ ]:
import gzip
import csv
from itertools import islice
import re
import matplotlib.pyplot as plt
import nltk
from nltk.tokenize import word_tokenize
import sys
import os
import numpy as np
import pandas as pd
import seaborn as sns
import random
import pickle
import math
import argparse
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as dt
from torch.autograd import Variable
from tqdm import tqdm_notebook as tqdm
from sklearn.metrics import roc_auc_score as roc
import matplotlib
matplotlib.use('Agg')

In [ ]:
%matplotlib inline


###Data Download Instructions



To download NOTEEVENTS.csv, ADMISSION.csv, and DIAGNOSES_ICD.csv you will need to request access and download the files from this link:https://physionet.org/content/mimiciii/1.4/

You will aslo need to install QuickUMLS which require file access from the National Library of Medicine, see instructions : https://github.com/Georgetown-IR-Lab/QuickUMLS

##Preprocessing Code

Path Variables

In [ ]:
DATA_PREPROCESSING_PATH = '/content/drive/MyDrive/CS598-DLH-Final-Project/Data-Preprocessing/'
NOTEEVENTS_CSV_GZ = DATA_PREPROCESSING_PATH + 'Original-Data/NOTEEVENTS.csv.gz'
outpath = DATA_PREPROCESSING_PATH + 'Output-Data/'


In [ ]:
NOTEEVENTS_CSV = DATA_PREPROCESSING_PATH + 'Original-Data/NOTEEVENTS.csv'

In [ ]:
ADMISSIONS_CSV = DATA_PREPROCESSING_PATH + 'Original-Data/ADMISSIONS.csv'

In [ ]:
DIAGNOSES_ICD = DATA_PREPROCESSING_PATH + 'Original-Data/DIAGNOSES_ICD.csv'

### Visualization of Data Statistics

Notes Events Data

In [ ]:
# Set size and color for plots
sns.reset_defaults()
sns.set(
    rc={'figure.figsize':(10,6)}, 
    style="white"
)

In [ ]:
noteevents = pd.read_csv(NOTEEVENTS_CSV, low_memory = False)

In [ ]:
noteevents_orig = noteevents

In [ ]:
noteevents.info()

In [ ]:
lns = [len(str(x)) for x in noteevents['TEXT']]
sns.distplot(lns, kde=False, axlabel='Document length')
plt.show()

In [ ]:
# Sort lengths
lns.sort()
# Take 5% as the removal size
rm_size = int(len(lns) / 100) * 5

# Now plot with removal of most/least frequent
sns.distplot(lns[rm_size:-rm_size], kde=False, axlabel='Document length')
plt.show()

In [ ]:
# Remove rows from the dataframe based on document length, this is not really
#straightforward, so we'll approximate it and find the document length that is used as a cutoff 
min_ln = max(lns[0:rm_size])
max_ln = min(lns[-rm_size:])

noteevents = noteevents[[True if len(str(x)) > min_ln and len(str(x)) < max_ln else False for x in noteevents['TEXT']]]
noteevents.head()

In [ ]:
print(f"Length after cleaning : {len(noteevents)}")
print(f"Length of the original: {len(noteevents_orig)}")

In [ ]:
sns.distplot(noteevents['SUBJECT_ID'].value_counts().values, kde=False, axlabel='Documents per patient')
plt.show()

In [ ]:
# Again a bit of clean-up, let's remove the bottom/top 1% of patients based on the number of 
#documents they have. 
docs_per_pt = noteevents['SUBJECT_ID'].value_counts()
docs_per_pt_vals = docs_per_pt.values
docs_per_pt_vals.sort()
docs_per_pt_vals

In [ ]:
rm_size = int(len(docs_per_pt_vals) / 100) * 1
min_ln = max(docs_per_pt_vals[0:rm_size])
max_ln = min(docs_per_pt_vals[-rm_size:])

In [ ]:
min_ln

In [ ]:
keep_subject_id = set([k for k, v in docs_per_pt.items() if v > 10 and v < 20])
noteevents_rm_docs_per_pt = noteevents[[True if subject_id in keep_subject_id else False 
                  for subject_id in noteevents['SUBJECT_ID'].values]]
noteevents_rm_docs_per_pt.head()

In [ ]:
print(f"Length after cleaning : {len(noteevents_rm_docs_per_pt)}")
print(f"Length of the original: {len(noteevents_orig)}")

In [ ]:
sns.distplot(noteevents_rm_docs_per_pt['SUBJECT_ID'].value_counts().values, kde=False, axlabel='Documents per patient')
plt.show()

Extract Sample of Data

In [ ]:
subjects = pd.DataFrame(keep_subject_id)

In [ ]:
len(subjects)

In [ ]:
#sample from subject ids 
sub_samples = subjects.sample(frac=0.01, random_state=4132023)

In [ ]:
len(sub_samples)

In [ ]:
keep_samp = sub_samples.iloc[:,0].values.tolist()

In [ ]:
sampled_notes = noteevents_rm_docs_per_pt[[True if subject_id in keep_samp else False 
                  for subject_id in noteevents_rm_docs_per_pt['SUBJECT_ID'].values]]

In [ ]:
sns.distplot(sampled_notes['SUBJECT_ID'].value_counts().values, kde=False, axlabel='Documents per patient')
plt.show()

In [ ]:
sampled_notes = sampled_notes.set_index(['ROW_ID'])

In [ ]:
sampled_notes.info()

In [ ]:
sampled_notes.to_csv('/content/drive/MyDrive/CS598-DLH-Final-Project/Data-Preprocessing/Original-Data/SAMPLEDNOTEEVENTS.csv', index=False)

In [ ]:
SAMPLEDNOTEEVENTS = '/content/drive/MyDrive/CS598-DLH-Final-Project/Data-Preprocessing/Original-Data/SAMPLEDNOTEEVENTS.csv'

In [ ]:
test = pd.read_csv(SAMPLEDNOTEEVENTS, low_memory=False)
test.info()

###Data Cleaning & DL Preparation

Call the noteEvents_preproc Python file

In [ ]:
!python3 /content/drive/MyDrive/CS598-DLH-Final-Project/patient_trajectory_prediction/data_cleaning/noteEvents_preproc.py

In [ ]:
OUTPUT = '/content/drive/MyDrive/CS598-DLH-Final-Project/Data-Preprocessing/Output-Data/output.csv'

In [ ]:
outputdata = pd.read_csv(OUTPUT, low_memory=False)

In [ ]:
outputdata.reset_index(inplace=True)
outputdata = outputdata.rename(columns = {'index':'new column name'})

In [ ]:
outputdata.head()

In [ ]:
outputdata.to_csv('/content/drive/MyDrive/CS598-DLH-Final-Project/QuickUMLS/data/chunkssmall/1.csv', index = False, header = False)

Call the quickUMLS_getCUI.py file

In [ ]:
!python3 /content/drive/MyDrive/CS598-DLH-Final-Project/patient_trajectory_prediction/concept_annotation/quickUMLS_getCUI.py

Call the quickumls_processing.py file

In [ ]:
!python3 /content/drive/MyDrive/CS598-DLH-Final-Project/patient_trajectory_prediction/concept_annotation/quickumls_processing.py /content/drive/MyDrive/CS598-DLH-Final-Project/QuickUMLS/data/outputchunkssmall/concatenated_output.csv

In [ ]:
test = pd.read_csv('/content/drive/MyDrive/CS598-DLH-Final-Project/Data-Preprocessing/Output-Data/post_processed_output.csv', low_memory=False,header = None)
test.iloc[:,2] = test.iloc[:,2].fillna(0)
test.iloc[:,2] = test.iloc[:,2].astype(int)
test.info()


In [ ]:
test.to_csv('/content/drive/MyDrive/CS598-DLH-Final-Project/Data-Preprocessing/Output-Data/post_process_output_no_index.csv', header = False, index = False)

Call the 01_data_prepartion.py file

In [ ]:
!python3 /content/drive/MyDrive/CS598-DLH-Final-Project/patient_trajectory_prediction/PyTorch_scripts/diagnoses_prediction/01_data_preparation.py

##Training Code, Evaluation Code, Pretrained Model

In [ ]:
!python3 /content/drive/MyDrive/CS598-DLH-Final-Project/patient_trajectory_prediction/PyTorch_scripts/diagnoses_prediction/02_FFN_diagprediction.py

In [ ]:
!python3 /content/drive/MyDrive/CS598-DLH-Final-Project/patient_trajectory_prediction/PyTorch_scripts/diagnoses_prediction/02_GRU_train_GPU.py

In [ ]:
!python3 /content/drive/MyDrive/CS598-DLH-Final-Project/patient_trajectory_prediction/PyTorch_scripts/diagnoses_prediction/03_GRU_test.py

## Ablation Study

Arguments

In [ ]:
class args:
  inputdata = '/content/drive/MyDrive/CS598-DLH-Final-Project/Data-Preprocessing/Output-Data/prepared_data.npz'
  outfile = '/content/drive/MyDrive/CS598-DLH-Final-Project/Data-Preprocessing/Output-Data/model_output_ablation.pt'
  hiddenDimSize = 50
  batchSize = 1000
  nEpochs = 10000
  lr = 0.01
  dropOut = 0.5
  kFold = 5
  withCCS = 0

ARGS = args()

In [ ]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()

        # Inputs to hidden layer linear transformation
        ARGS.inputdim = ARGS.numberOfInputCUIInts + ARGS.numberOfInputCCSInts if ARGS.withCCS else ARGS.numberOfInputCUIInts
        self.hidden = nn.Linear(ARGS.inputdim, ARGS.hiddenDimSize)
        self.hidden2 = nn.Linear(ARGS.hiddenDimSize, ARGS.numberOfOutputCodes)

        # Define sigmoid activation and softmax output
        # self.sigmoid = nn.Sigmoid()
        self.relu = nn.ReLU()

        # DropOut
        self.dropout = nn.Dropout(p=ARGS.dropOut)

    def forward(self, x):
        # Pass the input tensor through each of our operations
        x = self.hidden(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.hidden2(x)
        return x


class my_dataset(dt.Dataset):
    def __init__(self, data, label):
        self.data = data
        self.label = label

    def __getitem__(self, index):
        return self.data[index], self.label[index]

    def __len__(self):
        return len(self.data)


def load_tensors():
    # -----------------------------------------
    # pickle input map - each entry is a pair (subject_id, [(hadm_id,admittime, [CUIsvector], [CCSsvector])]
    subjecttoadm_map = pickle.load(open(ARGS.inputdata, 'rb'))
    setOfDistinctCUIs = set()
    setOfDistinctCCSs = set()
    cuitoint = dict()
    ccstoint = dict()
    for subject in subjecttoadm_map.keys():
        patientData = subjecttoadm_map[subject]
        for ithAdmis in patientData:
            for CUIcode in ithAdmis[2]:
                setOfDistinctCUIs.add(CUIcode)
            for CCScode in ithAdmis[3]:
                setOfDistinctCCSs.add(CCScode)
    for i, cui in enumerate(setOfDistinctCUIs):
        cuitoint[cui] = i
    for i, ccs in enumerate(setOfDistinctCCSs):
        ccstoint[ccs] = i
    print("-> " + str(
        len(subjecttoadm_map)) + " patients' CUI notes and CCS codes at dimension 0 for file: " + ARGS.inputdata)
    ARGS.numberOfInputCUIInts = len(setOfDistinctCUIs)
    ARGS.numberOfInputCCSInts = len(setOfDistinctCCSs)
    # -------------------------------------------
    ARGS.numberOfOutputCodes = len(setOfDistinctCCSs)
    print('Remaining patients:', len(subjecttoadm_map))
    # Convert everything to list of list of list (patient x admission x CUInote_vector/diagnoses to ease the manipulation in batches
    vectors_trainListX = []
    diagnoses_trainListY = []
    # hadm_id_List = []
    for pID, adList in subjecttoadm_map.items():
        for i, adm in enumerate(adList):
            # hadm_id_List.append(adm[0])
            if i + 1 == len(adList):
                # Avoid adding the last admission in X
                one_hot_CCS = [0] * ARGS.numberOfInputCCSInts
                for ccs_int in adm[3]:
                    one_hot_CCS[ccstoint[ccs_int]] = 1
                diagnoses_trainListY.append(one_hot_CCS)
                continue
            one_hot_CUI = [0] * ARGS.numberOfInputCUIInts
            one_hot_CCS = [0] * ARGS.numberOfInputCCSInts
            for cui_int in adm[2]:
                one_hot_CUI[cuitoint[cui_int]] = 1
            for ccs_int in adm[3]:
                one_hot_CCS[ccstoint[ccs_int]] = 1
            one_hot_X = one_hot_CUI + one_hot_CCS if ARGS.withCCS else one_hot_CUI
            vectors_trainListX.append(one_hot_X)
            if i != 0:
                # Add every admission diagnoses in Y but the first one's diagnoses
                diagnoses_trainListY.append(one_hot_CCS)

    mapIndexPosition = list(zip(vectors_trainListX, diagnoses_trainListY))
    random.shuffle(mapIndexPosition)
    vectors_trainListX, diagnoses_trainListY = zip(*mapIndexPosition)
    return vectors_trainListX, diagnoses_trainListY


def split(a, n):
    k, m = divmod(len(a), n)
    return (a[i * k + min(i, m):(i + 1) * k + min(i + 1, m)] for i in range(n))


def init_weights(m):
    if type(m) == nn.Linear:
        torch.nn.init.xavier_uniform(m.weight)
        m.bias.data.fill_(0.01)


def train():
    data_x, data_y = load_tensors()

    sizedata = len(data_x)
    print("Data of size:", sizedata)
    # Split dataset into 5 sub-datasets
    splitted_x = list(split(data_x, 5))
    splitted_y = list(split(data_y, 5))
    print("Available GPU :", torch.cuda.is_available())
    torch.cuda.set_device(0)
    k = ARGS.kFold

    # Prepare array of scores
    precision_list = []
    recall_list = []
    # valloss_list = []
    AUC_list = []
    for ind_i in range(0,k):
        # Prepare X_train Y_train X_test Y_test
        X_test = splitted_x[ind_i]
        Y_test = splitted_y[ind_i]
        # Deep copy, otherwise iteration problem
        copysplitX = list(splitted_x)
        copysplitY = list(splitted_y)
        del copysplitX[ind_i]
        del copysplitY[ind_i]
        X_train = copysplitX
        Y_train = copysplitY
        model = Network().cuda()
        # XAVIER Init
        model.apply(init_weights)
        with torch.cuda.device(0):
            # Hyperparameters :
            epochs = ARGS.nEpochs
            batchsize = ARGS.batchSize
            learning_rate = ARGS.lr
            log_interval = 2
            criterion = nn.BCEWithLogitsLoss()
            # criterion = nn.BCELoss()
            # criterion = nn.CrossEntropyLoss()
            optimizer = optim.SGD(model.parameters(), lr=learning_rate)
            # optimizer = optim.Adam(model.parameters(), lr=learning_rate)

            # Train loader
            numplist = np.array(X_train)
            arrX = np.concatenate(numplist).tolist()
            tensor_x = torch.Tensor(arrX).cuda()
            numplist = np.array(Y_train)
            arrY = np.concatenate(numplist).tolist()
            tensor_y = torch.Tensor(arrY).cuda()
            print("Shape X:", np.shape(arrX), "Shape Y:", np.shape(arrY))
            # tensor_x = torch.Tensor(np.array(X_train).tolist()).cuda()  # transform to torch tensor
            # tensor_y = torch.Tensor(np.array(Y_train).tolist()).cuda()
            dataset = dt.TensorDataset(tensor_x, tensor_y)  # create your dataset
            # train_size = int(len(dataset))
            # print("train_size =", train_size)
            train_loader = dt.DataLoader(
                dataset,
                batch_size=batchsize,
                shuffle=True)

            # Test loader
            tensor_x = torch.Tensor(np.array(X_test).tolist()).cuda()  # transform to torch tensor
            tensor_y = torch.Tensor(np.array(Y_test).tolist()).cuda()
            dataset = dt.TensorDataset(tensor_x, tensor_y)  # create your dataset
            test_loader = dt.DataLoader(
                dataset,
                batch_size=batchsize,
                shuffle=True)

            # Training
            for epoch in range(epochs):
                for batch_idx, (data, target) in enumerate(train_loader):
                    data, target = Variable(data), Variable(target)
                    optimizer.zero_grad()
                    net_out = model(data)
                    loss = criterion(net_out, target)
                    loss.backward()
                    optimizer.step()
                    if batch_idx % log_interval == 0:
                        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: '.format(
                            epoch, batch_idx * len(data), len(train_loader.dataset),
                                   100. * batch_idx / len(train_loader)))
                        print(loss.data)

            # saving model
            # torch.save(model.state_dict(), ARGS.outFile + str(ind_i))

            # Testing and save score
            total = 0
            correct = 0
            model.eval()
            # Validation loss
            # loss_values = []
            itr_ctr = 0
            for batch_idx, (data, target) in enumerate(test_loader):
                #with torch.no_grad():
                itr_ctr += 1
                data, target = Variable(data, volatile=True), Variable(target, volatile=True)
                net_out = model(data)
                loss = criterion(net_out, target)
                # loss_values.append(loss)

            # Validation Loss in the list
            # valloss_list.append(np.mean(loss_values))

            P = list()
            R = list()
            # Precisions
            for i in range(1,4):
                for data in test_loader:
                    x, labels = data
                    outputs = model(Variable(x)).detach() # output is a tensor of size [BATCHSIZE][ARGS.numberOfOutputCodes]
                    _, predicted = torch.topk(outputs.data, i)
                    for y_predlist, y in zip(predicted, labels):
                        for y_pred in y_predlist:
                            total += 1
                            if y[y_pred] == 1:
                                correct += 1

                precision = correct / total
                P.append(precision)
                correct = 0
                total = 0

            # Number of diagnostic for each sample (mean of 12 codes, max of 30 codes, R@10 - R@20 - R@30 seems appropriate)
            total_true_list = list()
            for data in test_loader:
                x, labels = data
                for y in labels :
                    total_true = 0
                    for val in y :
                        if val == 1:
                            total_true += 1
                    total_true_list.append(total_true)

            # Recalls
            for i in range(10,40,10):
                total_true_list_cpy = list(total_true_list)
                for data in test_loader:
                    x, labels = data
                    outputs = model(Variable(x)).detach()
                    _, predicted = torch.topk(outputs.data, i)
                    for y_predlist, y in zip(predicted, labels):
                        total += total_true_list_cpy.pop(0)
                        for y_pred in y_predlist:
                            if y[y_pred] == 1:
                                correct += 1

                recall = correct / total
                R.append(recall)
                correct = 0
                total = 0
            precision_list.append(P)
            recall_list.append(R)

            # AUROC
            YTRUE = None
            YPROBA = None
            for data in test_loader:
                x, labels = data
                x, labels = Variable(x), Variable(labels)
                outputs = model(x).detach().cpu().numpy()
                labels = labels.detach().cpu().numpy()
                for batch_true, batch_prob in zip(labels, outputs):
                    YTRUE = np.concatenate((YTRUE, [batch_true]), axis=0) if YTRUE is not None else [batch_true]
                    YPROBA = np.concatenate((YPROBA, [batch_prob]), axis=0) if YPROBA is not None else [batch_prob]
            ROC_avg_score=roc(YTRUE, YPROBA, average='micro', multi_class='ovr')
            AUC_list.append(ROC_avg_score)

    # Output score of each fold + average
    print("Scores for each fold:")
    print("Precision:", precision_list)
    print("Recall:", recall_list)
    print("AUROC:", AUC_list)
    # print("Loss:", valloss_list)
    print("Average scores:")
    P1=(sum([precision_list[k][0] for k in range(0, k)])/k)
    P2=(sum([precision_list[k][1] for k in range(0, k)])/k)
    P3=(sum([precision_list[k][2] for k in range(0, k)])/k)
    R10=(sum([recall_list[k][0] for k in range(0, k)])/k)
    R20=(sum([recall_list[k][1] for k in range(0, k)])/k)
    R30=(sum([recall_list[k][2] for k in range(0, k)])/k)
    AUROC=(sum([AUC_list[k] for k in range(0,k)])/k)
    # loss_avg=sum(valloss_list)/len(valloss_list)
    print("Precision@1:", P1)
    print("Precision@2:", P2)
    print("Precision@3:", P3)
    print("Recall@10:", R10)
    print("Recall@20:", R20)
    print("Recall@30:", R30)
    print("AUROC:", AUROC)
    # print("Loss:", loss_avg)

train()